<a href="https://colab.research.google.com/github/PingPingE/Learn_ML_DL/blob/main/Practice/Hands_On_ML/ch16-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import pandas as pd

# 감성 분석
- 데이터셋: IMDb 리뷰 데이터셋(자연어 처리계의 'hello world')
  - IMDb: Internet Movie Database


In [ ]:
(X_train, y_train), (X_test, y_test) = keras.datasets.imdb.load_data()

<string>:6: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:159: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  x_train, y_train = np.array(xs[:idx]), np.array(labels[:idx])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/datasets/imdb.py:160: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 

In [ ]:
X_train.shape

(25000,)

In [ ]:
X_test.shape

(25000,)

In [ ]:
X_train[0][:10]

[1, 14, 22, 16, 43, 530, 973, 1622, 1385, 65]

-----
- 이미 전처리 되어있음
- X_train은 리뷰들의 리스트이고, 각 리뷰는 넘파이 정수 배열로 표현
- 각 정수는 하나의 단어
  - 처리 과정
    - 구두점 제거
    - 모두 소문자
    - 공백으로 나누어 빈도에 따라 인덱싱
  - 정수 0,1,2는 알 수 없는 단어를 의미

In [ ]:
word_index=keras.datasets.imdb.get_word_index()
id_to_word = {id_ +3: word for word, id_ in word_index.items()}

In [ ]:
len(id_to_word) #===단어 개수

88584

In [ ]:
id_to_word[123] #===무작위로 함 추출해봄

'show'

In [ ]:
id_to_word[456]

'lives'

In [ ]:
id_to_word[4]

'the'

- 0,1,2에 단어 추가 해보기

In [ ]:
for id_, token in enumerate(("<pad>", "<sos>", "<unk>")):
  id_to_word[id_]=token

In [ ]:
print(id_to_word[0],id_to_word[1], id_to_word[2])

<pad> <sos> <unk>


- X_train 일부 디코딩

In [ ]:
" ".join([id_to_word[id_] for id_ in X_train[0][:10]])

'<sos> this film was just brilliant casting location scenery story'

## 텍스트 전처리 방법
- 보통 단어를 인코딩할 때는 구두점, 줄바꿈, 탭 등 많은 글자가 제외됨
- 그리고 공백을 사용해 단어 경계를 구분함
- 하지만 이게 최선은 아님
  - 중국어는 단어 사이에 공백을 사용하지 X
  - 독일어는 여러 단어를 공백 없이 붙여씀
  - 영어도 'San Francisco', 'ILoveDeepLearning'같은 경우가 있음
- 그럼 어떻게?
  - 다쿠 구도가 제안: <strong>부분 단어</strong> 수준으로 텍스트를 토큰화 or 복원하는 비지도 학습 방법
    - 이 방법은 공백을 하나의 문자로 취급하므로 언어 독립적임

In [ ]:
import tensorflow_datasets as tfds
datasets, info = tfds.load("imdb_reviews", as_supervised=True, with_info=True)
train_size=info.splits["train"].num_examples

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete3LHCK6/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete3LHCK6/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incomplete3LHCK6/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


###기본 전처리 함수

In [ ]:
def preprocess(X_batch, y_batch):
  X_batch = tf.strings.substr(X_batch,0,300) #===처음 300글자만 남김 (substring)
  #===정규식을 사용해서 일부 공백으로 변환
  X_batch= tf.strings.regex_replace(X_batch, b"<br\\s*/?>", b" ")
  X_batch= tf.strings.regex_replace(X_batch, b"[^a-zA-Z']", b" ")
  #===리뷰를 공백으로 나눔 -> ragged tensor(중첩 가변 길이 list)반환
  X_batch=tf.strings.split(X_batch)
  return X_batch.to_tensor(default_value=b"<pad>"), y_batch #===밀집 텐서로 바꾸고 동일한 길이가 되도록 패딩 토큰 "<pad>"로 모든 리뷰 패딩 

### 어휘 사전 구축

In [ ]:
from collections import Counter
vocabulary =Counter()
for X_batch, y_batch in datasets["train"].batch(32).map(preprocess):
  for review in X_batch:
    vocabulary.update(list(review.numpy()))

In [ ]:
vocabulary.most_common()[:3] #=== 가장 많이 등장하는 단어 세 개

[(b'<pad>', 214309), (b'the', 61137), (b'a', 38564)]

- 일부만 추출

In [ ]:
vocab_size=10000
truncated_vocabulary=[ word for word, count in vocabulary.most_common()[:vocab_size]]

### 각 단어 인덱싱
  - 13장에서 한 것 처럼 oov(out-of-vocabulary)버킷을 사용하는 룩업 테이블을 만들

In [ ]:
words=tf.constant(truncated_vocabulary)
word_ids=tf.range(len(truncated_vocabulary), dtype=tf.int64)
vocab_init=tf.lookup.KeyValueTensorInitializer(words, word_ids)
num_oov_buckets=1000
table=tf.lookup.StaticVocabularyTable(vocab_init, num_oov_buckets)

In [ ]:
table.lookup(tf.constant([b"This movie was faaaaaaantastic".split()]))

<tf.Tensor: shape=(1, 4), dtype=int64, numpy=array([[   22,    12,    11, 10770]])>

------
-  'faaaaaaantastic'은 룩업 테이블에 없어서 oov버킷(10000~11000)에 매핑된 것을 확인 

## 최종 훈련 세트 만들기
1. 리뷰를 배치로 묶고
2. preprocess()함수를 사용해 짧은 시퀀스로 바꾸기
3. 룩업 테이블을 사용해 단어 인코딩
4. 배치 프리페치 

In [ ]:
def encode_words(X_batch, y_batch):
  return table.lookup(X_batch), y_batch


In [ ]:
train_set= datasets["train"].batch(32).map(preprocess)
train_set=train_set.map(encode_words).prefetch(1)

## 모델 생성 및 훈련

In [ ]:
embed_size=128 #==벡터 크기(하이퍼파라미터)
model=keras.models.Sequential([
                               keras.layers.Embedding(vocab_size+num_oov_buckets, embed_size, input_shape=[None]), #=== input: [배치 크기, 타임 스텝 수], output: [배치 크기, 타임 스텝 수, 임베딩 크기]
                               keras.layers.GRU(128, return_sequences=True),
                               keras.layers.GRU(128),
                               keras.layers.Dense(1,activation='sigmoid')
])


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 128)         1408000   
_________________________________________________________________
gru (GRU)                    (None, None, 128)         99072     
_________________________________________________________________
gru_1 (GRU)                  (None, 128)               99072     
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 1,606,273
Trainable params: 1,606,273
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
history=model.fit(train_set, epochs=5)

Epoch 1/5
782/782 [==============================] - 100s 123ms/step - loss: 0.6940 - accuracy: 0.5116
Epoch 2/5
782/782 [==============================] - 95s 121ms/step - loss: 0.5232 - accuracy: 0.7413
Epoch 3/5
782/782 [==============================] - 95s 122ms/step - loss: 0.3304 - accuracy: 0.8593
Epoch 4/5
782/782 [==============================] - 95s 122ms/step - loss: 0.2357 - accuracy: 0.9085
Epoch 5/5
782/782 [==============================] - 96s 123ms/step - loss: 0.1656 - accuracy: 0.9399


### 사전훈련된 임베딩 재사용하기
- 위처럼 단어 임베딩을 직접 훈련할 수도 있지만, 사전 훈련된 임베딩을 재사용할 수도 있음
- [TF 허브 저장소](https://tfhub.dev/)에서 원하는 모듈을 찾아서 활용하면됨
  - tensorflow_hub.KerasLayer 층이 모듈과 사전 훈련된 가중치를 자동으로 다운해서 모델에 포함시켜줌
    - 해당 층은 훈련되지 않음(근데 trainable=True하면 미세 조정할 순 있음)

In [2]:
import tensorflow_hub as hub
model_hub= keras.Sequential([
                             hub.KerasLayer("https://tfhub.dev/google/tf2-preview/nnlm-en-dim50/1", dtype=tf.string, input_shape=[], output_shape=[50]),
                             keras.layers.Dense(128, activation="relu"),
                             keras.layers.Dense(1,activation="sigmoid")
                             
])

In [3]:
model_hub.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 50)                48190600  
_________________________________________________________________
dense (Dense)                (None, 128)               6528      
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 48,197,257
Trainable params: 6,657
Non-trainable params: 48,190,600
_________________________________________________________________


# 신경망 기계 번역(NMT)을 위한 인코더-디코더 네트워크
<img src="https://user-images.githubusercontent.com/13735079/34563713-d9b26bb6-f13a-11e7-8c12-a1830a397d78.png"/>

- 책에 있는 그림(위 그림)을 보면 인코더 input 주입 시에 <strong>문장을 뒤집어서</strong> 넣는 것을 알 수 있는데,
 - 책에선 '문장의 시작 부분을 인코더에 마지막으로 주입한다. 디코더가 번역할 첫 부분이기 때문이다'라고 하고 있음
- 근데 인터넷을 찾아보면 뒤집어서 주입하는 경우를 찾아볼 수 없었음
- 그 이유는, <strong>인코더에 어떤 RNN</strong>을 쓰느냐에 따라 다르기 때문
  - <strong>Bi-Directional RNN</strong>
    - 왼쪽 -> 오른쪽 
    - 오른쪽 <- 왼쪽<br><br>
    <img src="https://miro.medium.com/max/1088/1*sBZq013xGoFvyH5e0sXtdg.png"/>
<br><br>
  - <strong>Uni-directional RNN</strong>
    - 왼쪽 -> 오른쪽
- 즉, 위 그림은 Uni-directional RNN이라서 뒤집어서 넣는게 의미가 있었던 것이고,
- 주로 NMT에서 많이 쓰는건 Bi-Directional RNN이라서 인터넷에 굳이 input을 뒤집은게 안보인거였음

참조)
- [github_issues](https://github.com/tensorflow/nmt/issues/231)

## 기본적인 인코더 - 디코더 모델

In [6]:
pip install tensorflow_addons

     |████████████████████████████████| 706kB 5.7MB/s 


In [22]:
import tensorflow_addons as tfa

vocab_size, embed_size=10000, 128
encoder_inputs=keras.layers.Input(shape=[None], dtype=np.int32)
decoder_inputs=keras.layers.Input(shape=[None], dtype=np.int32)
sequence_lengths=keras.layers.Input(shape=[], dtype=np.int32)

embeddings=keras.layers.Embedding(vocab_size, embed_size)
encoder_embeddings= embeddings(encoder_inputs)
decoder_embeddings=embeddings(decoder_inputs)

encoder=keras.layers.LSTM(512, return_state=True)
encoder_output, state_h, state_c = encoder(encoder_embeddings)
encoder_state=[state_h, state_c]

sampler=tfa.seq2seq.sampler.TrainingSampler()

decoder_cell=keras.layers.LSTMCell(512)
output_layer=keras.layers.Dense(vocab_size)
decoder=tfa.seq2seq.basic_decoder.BasicDecoder(decoder_cell, sampler, output_layer=output_layer)
final_outputs, final_state, final_sequence_lengths = decoder(
    decoder_embeddings, 
    initial_state=encoder_state,
    sequence_length=sequence_lengths)

Y_proba = tf.nn.softmax(final_outputs.rnn_output)

model2 = keras.models.Model(
    inputs=[encoder_inputs, decoder_inputs, sequence_lengths],
    outputs=[Y_proba])

In [23]:
model2.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_38 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
input_37 (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
embedding_12 (Embedding)        (None, None, 128)    1280000     input_37[0][0]                   
                                                                 input_38[0][0]                   
__________________________________________________________________________________________________
lstm_11 (LSTM)                  [(None, 512), (None, 1312768     embedding_12[0][0]         